In [ ]:
# Cleanup
!sudo rm -rf TCPMSiameseBERT
!pip freeze | grep -v "^-e" | xargs pip uninstall -y


# Python Setup
# Install Python 3.10
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-dev python3.10-distutils -y

# Install pip for Python 3.10
!wget https://bootstrap.pypa.io/get-pip.py
!python3.10 get-pip.py

# Symlink python and pip to python3.10
!sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.10 1
!sudo update-alternatives --install /usr/bin/pip pip /usr/local/bin/pip3.10 1
!sudo update-alternatives --set python /usr/bin/python3.10
!sudo update-alternatives --set python3 /usr/bin/python3.10
!nvcc --version

ERROR: Invalid requirement: '@': Expected package name at the start of dependency specifier
    @
    ^
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,765 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,743 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,021 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://pp

In [ ]:
####################### CUDA Setup #######################
# Step 1: Download the pin file that prioritizes the CUDA local repo over others in APT
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin

# Step 2: Move the pin file into APT's preferences directory to enforce priority
!mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600

# Step 3: Download the CUDA 12.2 local repository installer for Ubuntu 22.04
!wget https://developer.download.nvidia.com/compute/cuda/12.2.0/local_installers/cuda-repo-ubuntu2204-12-2-local_12.2.0-535.54.03-1_amd64.deb

# Step 4: Install the local CUDA repository .deb package
!dpkg -i cuda-repo-ubuntu2204-12-2-local_12.2.0-535.54.03-1_amd64.deb

# Step 5: Copy the GPG key from the repo to APT's keyring location to allow secure installation
!cp /var/cuda-repo-ubuntu2204-12-2-local/cuda-*-keyring.gpg /usr/share/keyrings/

# Step 6: Update the package index to include packages from the newly added local CUDA repo
!apt-get update

# Step 7: Install all core CUDA 12.2 components (compiler, libraries, runtime)
!apt-get install cuda-12-2

# Step 8: Export the CUDA path environment variable (temporary in Colab session)
!export CUDA_PATH=/usr/local/cuda-12.2/

# Step 9: Use update-alternatives to point the system’s default 'cuda' path to CUDA 12.2
!update-alternatives --set cuda /usr/local/cuda-12.2


####################### cuDNN Setup #######################
# Remove conflicting APT source
!rm -f /etc/apt/sources.list.d/cudnn-local.list
!apt-get remove -y --purge libcudnn9 libcudnn9-dev libcudnn9-cuda-12 libcudnn9-dev-cuda-12

# Step 1: Download the cuDNN 9.3.0 local repo
!wget https://developer.download.nvidia.com/compute/cudnn/9.3.0/local_installers/cudnn-local-repo-ubuntu2204-9.3.0_1.0-1_amd64.deb

# Step 2: Install the local repo .deb
!dpkg -i cudnn-local-repo-ubuntu2204-9.3.0_1.0-1_amd64.deb

# Step 3: Copy the correct GPG key that matches the auto-configured source
!cp /var/cudnn-local-repo-ubuntu2204-9.3.0/cudnn-local-*-keyring.gpg /usr/share/keyrings/
!cp /var/cudnn-local-repo-ubuntu2204-9.3.0/cudnn-local-D9334AA3-keyring.gpg /usr/share/keyrings/

# Step 4: Update package lists using the registered repo
!apt-get update

# Step 5: Install cuDNN 9.3.1 binaries
!apt-get install -y --allow-change-held-packages libcudnn9-cuda-12=9.3.0.75-1 libcudnn9-dev-cuda-12=9.3.0.75-1

####################### Verification Step #######################
!nvcc --version
!apt-cache policy libcudnn9-cuda-12
!echo "Finished apt-cache policy"
!dpkg -L libcudnn9-cuda-12 | grep libcudnn.so

# Symlink
!ln -sf /usr/lib/x86_64-linux-gnu/libcudnn.so.9.3.0 /usr/local/cuda-12.2/lib64/libcudnn.so.9
!ln -sf /usr/local/cuda-12.2/lib64/libcudnn.so.9 /usr/local/cuda-12.2/lib64/libcudnn.so

import os
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-12.2/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")

--2025-06-07 18:57:04--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.45.46.203, 23.45.46.200
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.45.46.203|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu2204.pin’

cuda-ubuntu2204.pin 100%[===================>]     190  --.-KB/s    in 0s      

2025-06-07 18:57:04 (143 MB/s) - ‘cuda-ubuntu2204.pin’ saved [190/190]

--2025-06-07 18:57:04--  https://developer.download.nvidia.com/compute/cuda/12.2.0/local_installers/cuda-repo-ubuntu2204-12-2-local_12.2.0-535.54.03-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.45.46.203, 23.45.46.200
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.45.46.203|:443... connected.
HTTP request sent, awaitin

In [ ]:
%cd /content/
!rm -rf TCPMSiameseBERT
!git clone --branch main https://github.com/Adir111/TCPMSiameseBERT.git
%cd TCPMSiameseBERT/PhaseB

# Install project dependencies with Python 3.10
!python3.10 -m pip install -r requirements.txt

import sys
sys.path.append("/content/TCPMSiameseBERT/PhaseB/bert_siamese_authorship_verification")

import os
os.environ.pop("MPLBACKEND", None)  # Clear inline backend
os.environ["TF_USE_LEGACY_KERAS"] = "1"

/content
Cloning into 'TCPMSiameseBERT'...
remote: Enumerating objects: 3953, done.
remote: Counting objects: 100% (594/594), done.
remote: Compressing objects: 100% (332/332), done.
remote: Total 3953 (delta 279), reused 446 (delta 262), pack-reused 3359 (from 2)
Receiving objects: 100% (3953/3953), 99.06 MiB | 30.77 MiB/s, done.
Resolving deltas: 100% (2044/2044), done.
Updating files: 100% (712/712), done.
/content/TCPMSiameseBERT/PhaseB
Ignoring tensorflow-gpu: markers 'sys_platform == "win32"' don't match your environment
Ignoring tensorflow-addons: markers 'sys_platform == "win32"' don't match your environment
Ignoring keras: markers 'sys_platform == "win32"' don't match your environment
Ignoring transformers: markers 'sys_platform == "win32"' don't match your environment
  Using cached scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 k

In [ ]:
%cd /content/TCPMSiameseBERT/PhaseB/bert_siamese_authorship_verification
!PYTHONPATH=/content/TCPMSiameseBERT python3.10 main.py

Streaming output truncated to the last 5000 lines.
25/25 [==============================] - 1s 44ms/step
wandb: [INFO] Signal representation: [0.375, 0.625, 0.625, 0.5, 0.375, 0.625, 0.375, 0.5, 0.375, 0.625, 0.625, 0.875, 0.75, 0.625, 0.625, 0.625, 0.5, 0.375, 0.25, 0.25, 0.5, 0.625, 0.625, 0.375, 0.375, 0.75, 0.875, 1.0, 0.5, 0.375, 0.25, 0.375, 0.375, 0.25, 0.5, 0.875, 0.75, 0.625, 0.75, 0.75, 0.5, 1.0, 0.875, 0.625, 0.625, 0.625, 0.5, 0.5, 0.625, 0.375, 0.625, 0.625, 0.625, 0.625, 0.625, 0.75, 0.875, 0.625, 0.5, 0.375, 0.5, 0.375, 0.625, 0.5, 0.5, 0.75, 0.375, 0.625, 0.375, 0.375, 0.5, 0.625, 0.75, 0.875, 0.625, 0.25, 0.25, 0.5, 0.75, 0.75, 0.625, 0.375, 0.75, 0.625, 0.875, 0.625, 0.25, 0.125, 0.5, 0.25, 0.625, 0.5, 0.5, 0.5, 0.25, 0.375, 0.75, 0.75, 0.875, 0.5]
wandb: [INFO] Signal generated for text: The Two Noble Kinsmen by Shakespeare.txt by model: CharlotteBronte_ElizabethCleghornGaskell
wandb: ✅ Logged Signal Representation for The Two Noble Kinsmen by Shakespeare.txt using C